In [ ]:
# for scraping
import random
import time
import os
import sys
import pickle
import re
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from selenium import webdriver
chromedriver = '/Applications/chromedriver'
os.environ["webdriver.chrome.driver"] = chromedriver

# for cleaning
import pandas as pd
import seaborn as sbn
import matplotlib.pyplot as plt
import datetime as dt

# for modeling
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV
from sklearn.metrics import r2_score

In [ ]:
sold_by_area_links_Seattle = [
    'https://www.redfin.com/city/16163/WA/Seattle/filter/include=sold-5yr,viewport=47.74409:47.69068:-122.30638:-122.40011',
    'https://www.redfin.com/city/16163/WA/Seattle/filter/include=sold-5yr,viewport=47.73611:47.68268:-122.2574:-122.35113',
    'https://www.redfin.com/city/16163/WA/Seattle/filter/include=sold-5yr,viewport=47.70017:47.67345:-122.31544:-122.3623', 
    'https://www.redfin.com/city/16163/WA/Seattle/filter/include=sold-5yr,viewport=47.7098:47.65635:-122.34018:-122.43391', 
    'https://www.redfin.com/city/16163/WA/Seattle/filter/include=sold-5yr,viewport=47.6748:47.62132:-122.30136:-122.39509', 
    'https://www.redfin.com/city/16163/WA/Seattle/filter/include=sold-5yr,viewport=47.69562:47.64215:-122.24063:-122.33435', 
    'https://www.redfin.com/city/16163/WA/Seattle/filter/include=sold-5yr,viewport=47.63781:47.61105:-122.31995:-122.36682', 
    'https://www.redfin.com/city/16163/WA/Seattle/filter/include=sold-5yr,viewport=47.67059:47.56356:-122.21313:-122.40058', 
    'https://www.redfin.com/city/16163/WA/Seattle/filter/include=sold-5yr,viewport=47.59167:47.56489:-122.37725:-122.42411', 
    'https://www.redfin.com/city/16163/WA/Seattle/filter/include=sold-5yr,viewport=47.5792:47.52562:-122.32803:-122.42176', 
    'https://www.redfin.com/city/16163/WA/Seattle/filter/include=sold-5yr,viewport=47.54585:47.49223:-122.31099:-122.40472',
    'https://www.redfin.com/city/16163/WA/Seattle/filter/include=sold-5yr,viewport=47.59386:47.5403:-122.26363:-122.35736', 
    'https://www.redfin.com/city/16163/WA/Seattle/filter/include=sold-5yr,viewport=47.5791:47.52552:-122.22798:-122.32171',
    'https://www.redfin.com/city/16163/WA/Seattle/filter/include=sold-5yr,viewport=47.53334:47.47971:-122.20693:-122.30066']

def scrape_sales_pages(salespages_list):
    """
    Takes in a list of complete urls for sold pages within a given area; fetches relative urls for listings.
    At the moment, Redfin will only go up to page 17 within a given geographical boundary.

    salespages_list: list of complete urls (redfin.com/...) as strings
    """
    sales_links = []
    salespages_missed = []
    salespages_completed = []
    error_list = []
    user_agent = UserAgent()
    for pagelink in salespages_list:
        for i in range(17):
            if i > 1:
                target_url = (pagelink + '/page-' + str(i))
            else:
                target_url = pagelink
            user_agent = {'User-agent': user_agent.random}
            try:
                response  = requests.get(target_url, headers = user_agent)
                time.sleep(2)
                soup = BeautifulSoup(response.content, 'html.parser')
                for pagelink in soup.find_all('a', class_ = 'slider-item hidden'):
                    sales_links.append(pagelink.get('href'))
            except:
                error_list.append(sys.exc_info()[0])
                error_list.append(response.status_code)
                salespages_missed.append(target_url)
            time.sleep(random.random()*11)
        salespages_completed.append(pagelink)
    # Geo areas overlap; dropping duplicates of urls
    return {'listing_links': list(set(sales_links)), 'pages_scraped': salespages_completed, 'pages_missed': salespages_missed}

listing_links = scrape_sales_pages(sold_by_area_links_Seattle)

In [ ]:
def scrape_listing_pages(sales_links, pickle_it = True):
    """
    Given a list of relative Redfin links (/WA/Seattle...), uses selenium and chromedriver to
    load listing pages, scroll down, and scrape page source. SLOW because of that but no catchpa 
    issues. Returns a list of scraped soup objects recast as strings at even indices and their 
    corresponding links at the subsequent odd indices.

    sales_links: input list of relative links to Redfin listings as strings
    pickle_it: should function regularly dump scrapes (True) or deliver all at end (False)
    listingpage_scrapes: output list of alternating scrapes as strings and urls as strings
    """
    base_url = 'https://www.redfin.com'
    listingpage_scrapes = []
    driver = webdriver.Chrome(chromedriver)
    counter = 1
    # Iterating through list of links
    for link in sales_links:
        listing_url = base_url + link
        driver.get(listing_url)
        for i in range(14):
            #Scroll
            driver.execute_script("window.scrollBy({top: 700,left: 0,behavior: 'smooth'});")
            time.sleep(0.5 + random.random()*1.5)
        # Getting the page source and soupifying it
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        listingpage_scrapes.append(soup)
        listingpage_scrapes.append(link)
        counter += 1
        # Intermittent pickling of raw html scrapes
        if pickle_it & counter%100 == 0:
            dump_list = [str(x) for x in listingpage_scrapes]
            with open(('listingpage_scrapes_dump' + str(counter) + '.txt'), 'wb') as file_pointer:
                pickle.dump(dump_list, file_pointer)
    if pickle_it:
        dump_list = [str(x) for x in listingpage_scrapes]
        with open(('listingpage_scrapes_dump' + str(counter) + '.txt'), 'wb') as file_pointer:
            pickle.dump(dump_list, file_pointer)
        return (str(counter) + ' done and pickled.')
    if not pickle_it:
        return listingpage_scrapes

# Scraping content from individual listing pages
listing_scrapes = scrape_listing_pages(listing_links)

In [ ]:
listing_scrapes = []
dumps_list = ['listingpage_scrapes_dump100', 'listingpage_scrapes_dump200', 
              'listingpage_scrapes_dump300', 'listingpage_scrapes_dump400', 'listingpage_scrapes_dump500', 
              'listingpage_scrapes_dump600', 'listingpage_scrapes_dump700', 'listingpage_scrapes_dump800', 
              'listingpage_scrapes_dump900', 'listingpage_scrapes_dump1000', 'listingpage_scrapes_dump1100', 
              'listingpage_scrapes_dump1200', 'listingpage_scrapes_dump1300', 'listingpage_scrapes_dump1400', 
              'listingpage_scrapes_dump1500', 'listingpage_scrapes_dump1600', 'listingpage_scrapes_dump1700', 
              'listingpage_scrapes_dump1800', 'listingpage_scrapes_dump1900', 'listingpage_scrapes_dump2000', 
              'listingpage_scrapes_dump2100', 'listingpage_scrapes_dump2200', 'listingpage_scrapes_dump2300', 
              'listingpage_scrapes_dump2400', 'listingpage_scrapes_dump2500', 'listingpage_scrapes_dump2600', 
              'listingpage_scrapes_dump2700', 'listingpage_scrapes_dump2800', 'listingpage_scrapes_dump2900', 
              'listingpage_scrapes_dump3112']

for i in dumps_list:
    with open((i+'.txt'), 'rb') as fp:
        listing_scrapes += pickle.load(fp)
        
for i in range(0, len(listing_scrapes), 2):
    listing_scrapes[i] = BeautifulSoup(listing_scrapes[i], 'html.parser')

def parse_sold_page(soup, url):
    """
    Grabs desired information from Redfin page for a single sold property using BeatifulSoup.
    Returns a dictionary of the desired information.
    """
    pull_digits = re.compile('[0-9.]+')
    property_dict = {}
    try:
        property_dict['address'] = soup.find('span', class_ = 'street-address').text
    except:
        property_dict['address'] = ''
    try:
        property_dict['ZIP'] = soup.find('span', class_ = 'postal-code').text
    except:
        property_dict['ZIP'] = ''
    try:
        property_dict['comm'] = soup.find(string = 'Community').find_next('span', class_ = 'content text-right').text
    except:
        property_dict['comm'] = ''
    try:
        property_dict['price'] = pull_digits.search(soup.find('div', class_ = 'info-block price').text.replace(',', '')).group()
    except:
        property_dict['price'] = ''
    try:
        property_dict['beds'] = pull_digits.search(soup.find(attrs = {'data-rf-test-id': "abp-beds"}).find('div', class_ = 'statsValue').text).group()
    except:
        property_dict['beds'] = ''
    try:
        property_dict['baths'] = pull_digits.search(soup.find(attrs = {'data-rf-test-id': "abp-baths"}).find('div', class_ = 'statsValue').text).group()
    except:
        property_dict['baths'] = ''
    try:
        property_dict['size'] = pull_digits.search(soup.find('div', class_ = 'info-block sqft').find('span', class_ = 'statsValue').text.replace(',', '')).group()
    except:
        property_dict['size'] = ''
    try:
        property_dict['style'] = soup.find(string = 'Style').find_next().text
    except:
        property_dict['style'] = ''
    try:
        property_dict['lot'] = pull_digits.search(soup.find(string = 'Lot Size').find_next().text.replace(',', '')).group()
    except:
        property_dict['lot'] = ''
    try:
        property_dict['age'] = soup.find(string = 'Year Built').find_next().text
    except:
        property_dict['age'] = ''
    try:
        property_dict['status'] = soup.find(attrs = {'data-rf-test-id': 'abp-status'}).find('span', class_ = 'value').text
    except:
        property_dict['status'] = ''
    try:
        property_dict['sold'] = soup.find('div', class_ = "Pill Pill--red padding-vert-smallest padding-horiz-smaller font-size-smaller font-weight-bold font-color-white HomeSash margin-top-smallest margin-right-smaller").text.replace('SOLD BY REDFIN ', '')
    except:
        property_dict['sold'] = ''
    try:
        property_dict['park'] = soup.find(string = 'Parking Information').find_next().text
    except:
        property_dict['park'] = ''
    try:
        property_dict['brok'] = pull_digits.search(soup.find(string = "Buyer's Brokerage Compensation").find_next('span', class_ = 'content text-right').text).group()
    except:
        property_dict['brok'] = ''
    property_dict['url'] = url
    return property_dict
    
data_dicts = []
for i in range(0, len(listing_scrapes), 2):
    data_dicts.append(parse_sold_page(listing_scrapes[i], listing_scrapes[i+1]))

listings_data = pd.DataFrame(data_dicts)

In [ ]:
# Not considering the houseboats, plexes, co-ops, and other oddballs (different enough to probably skew model)
type_dict = {'': '',
 '1 1/2 Story': 'house',
 '1 1/2 Story with Basement': 'house',
 '1 1/2 Story with Basement, Cape Cod': 'house',
 '1 1/2 Story with Basement, Colonial': 'house',
 '1 1/2 Story with Basement, Contemporary': 'house',
 '1 1/2 Story with Basement, Craftsman': 'house',
 '1 1/2 Story with Basement, Modern': 'house',
 '1 1/2 Story with Basement, Northwestern Contemporary': 'house',
 '1 1/2 Story with Basement, Traditional': 'house',
 '1 1/2 Story with Basement, Tudor': 'house',
 '1 1/2 Story, Cape Cod': 'house',
 '1 1/2 Story, Contemporary': 'house',
 '1 1/2 Story, Craftsman': 'house',
 '1 1/2 Story, Northwestern Contemporary': 'house',
 '1 1/2 Story, Other (See Remarks)': 'house',
 '1 1/2 Story, Traditional': 'house',
 '1 1/2 Story, Tudor': 'house',
 '1 Story': 'house',
 '1 Story with Basement': 'house',
 '1 Story with Basement, Cape Cod': 'house',
 '1 Story with Basement, Contemporary': 'house',
 '1 Story with Basement, Craftsman': 'house',
 '1 Story with Basement, Modern': 'house',
 '1 Story with Basement, Northwestern Contemporary': 'house',
 '1 Story with Basement, Other (See Remarks)': 'house',
 '1 Story with Basement, Spanish/Southwestern': 'house',
 '1 Story with Basement, Traditional': 'house',
 '1 Story with Basement, Tudor': 'house',
 '1 Story, Cabin': 'house',
 '1 Story, Cape Cod': 'house',
 '1 Story, Contemporary': 'house',
 '1 Story, Craftsman': 'house',
 '1 Story, Modern': 'house',
 '1 Story, Northwestern Contemporary': 'house',
 '1 Story, Other (See Remarks)': 'house',
 '1 Story, Traditional': 'house',
 '2 Stories with Basement': 'house',
 '2 Stories with Basement, Cape Cod': 'house',
 '2 Stories with Basement, Colonial': 'house',
 '2 Stories with Basement, Contemporary': 'house',
 '2 Stories with Basement, Craftsman': 'house',
 '2 Stories with Basement, Modern': 'house',
 '2 Stories with Basement, Northwestern Contemporary': 'house',
 '2 Stories with Basement, Other (See Remarks)': 'house',
 '2 Stories with Basement, Traditional': 'house',
 '2 Stories with Basement, Tudor': 'house',
 '2 Stories with Basement, Victorian': 'house',
 '2 Story': 'house',
 '2 Story, Cape Cod': 'house',
 '2 Story, Contemporary': 'house',
 '2 Story, Craftsman': 'house',
 '2 Story, Modern': 'house',
 '2 Story, Northwestern Contemporary': 'house',
 '2 Story, Other (See Remarks)': 'house',
 '2 Story, Spanish/Southwestern': 'house',
 '2 Story, Traditional': 'house',
 '4-Plex': '',
 '5-9 Units': '',
 'Co-op': '',
 'Condominium (2 Levels)': 'condo',
 'Condominium (2 Levels), Contemporary': 'condo',
 'Condominium (2 Levels), Loft': 'condo',
 'Condominium (2 Levels), Modern': 'condo',
 'Condominium (2 Levels), Townhouse': 'condo',
 'Condominium (2 Levels), Traditional': 'condo',
 'Condominium (3+ Levels)': 'condo',
 'Condominium (3+ Levels), Contemporary': 'condo',
 'Condominium (3+ Levels), Modern': 'condo',
 'Condominium (3+ Levels), Townhouse': 'condo',
 'Condominium (Single Level)': 'condo',
 'Condominium (Single Level), Contemporary': 'condo',
 'Condominium (Single Level), Craftsman': 'condo',
 'Condominium (Single Level), Loft': 'condo',
 'Condominium (Single Level), Modern': 'condo',
 'Condominium (Single Level), Other (See Remarks)': 'condo',
 'Condominium (Single Level), Spanish/Southwestern': 'condo',
 'Condominium (Single Level), Studio': 'condo',
 'Condominium (Single Level), Traditional': 'condo',
 'Condominium (Single Level), Tudor': 'condo',
 'Duplex': '',
 'Houseboat, Cabin': '',
 'Houseboat, Contemporary': '',
 'Manufactured Double-Wide': '',
 'Multi-Family': '',
 'Multi-Level': 'house',
 'Multi-Level, Contemporary': 'house',
 'Multi-Level, Craftsman': 'house',
 'Multi-Level, Modern': 'house',
 'Multi-Level, Northwestern Contemporary': 'house',
 'Multi-Level, Other (See Remarks)': 'house',
 'Multi-Level, Traditional': 'house',
 'Multi-Level, Tudor': 'house',
 'Multi-Level, Victorian': 'house',
 'Residential (1+ Acre)': 'house',
 'Residential (<1 Acre)': 'house',
 'Single Family Residential': 'house',
 'Split-Entry': 'house',
 'Split-Entry, Contemporary': 'house',
 'Split-Entry, Craftsman': 'house',
 'Split-Entry, Modern': 'house',
 'Split-Entry, Northwestern Contemporary': 'house',
 'Split-Entry, Other (See Remarks)': 'house',
 'Split-Entry, Traditional': 'house',
 'Townhouse': 'townhouse',
 'Townhouse, Contemporary': 'townhouse',
 'Townhouse, Craftsman': 'townhouse',
 'Townhouse, Modern': 'townhouse',
 'Townhouse, Northwestern Contemporary': 'townhouse',
 'Townhouse, Townhouse': 'townhouse',
 'Townhouse, Traditional': 'townhouse',
 'Tri-Level': 'house',
 'Tri-Level, Cape Cod': 'house',
 'Tri-Level, Contemporary': 'house',
 'Tri-Level, Craftsman': 'house',
 'Tri-Level, Modern': 'house',
 'Tri-Level, Northwestern Contemporary': 'house',
 'Tri-Level, Other (See Remarks)': 'house',
 'Tri-Level, Traditional': 'house',
 'Triplex': ''}
basement_dict = {'': 0,
 '1 1/2 Story': 0,
 '1 1/2 Story with Basement': 1,
 '1 1/2 Story with Basement, Cape Cod': 1,
 '1 1/2 Story with Basement, Colonial': 1,
 '1 1/2 Story with Basement, Contemporary': 1,
 '1 1/2 Story with Basement, Craftsman': 1,
 '1 1/2 Story with Basement, Modern': 1,
 '1 1/2 Story with Basement, Northwestern Contemporary': 1,
 '1 1/2 Story with Basement, Traditional': 1,
 '1 1/2 Story with Basement, Tudor': 1,
 '1 1/2 Story, Cape Cod': 0,
 '1 1/2 Story, Contemporary': 0,
 '1 1/2 Story, Craftsman': 0,
 '1 1/2 Story, Northwestern Contemporary': 0,
 '1 1/2 Story, Other (See Remarks)': 0,
 '1 1/2 Story, Traditional': 0,
 '1 1/2 Story, Tudor': 0,
 '1 Story': 0,
 '1 Story with Basement': 1,
 '1 Story with Basement, Cape Cod': 1,
 '1 Story with Basement, Contemporary': 1,
 '1 Story with Basement, Craftsman': 1,
 '1 Story with Basement, Modern': 1,
 '1 Story with Basement, Northwestern Contemporary': 1,
 '1 Story with Basement, Other (See Remarks)': 1,
 '1 Story with Basement, Spanish/Southwestern': 1,
 '1 Story with Basement, Traditional': 1,
 '1 Story with Basement, Tudor': 1,
 '1 Story, Cabin': 0,
 '1 Story, Cape Cod': 0,
 '1 Story, Contemporary': 0,
 '1 Story, Craftsman': 0,
 '1 Story, Modern': 0,
 '1 Story, Northwestern Contemporary': 0,
 '1 Story, Other (See Remarks)': 0,
 '1 Story, Traditional': 0,
 '2 Stories with Basement': 1,
 '2 Stories with Basement, Cape Cod': 1,
 '2 Stories with Basement, Colonial': 1,
 '2 Stories with Basement, Contemporary': 1,
 '2 Stories with Basement, Craftsman': 1,
 '2 Stories with Basement, Modern': 1,
 '2 Stories with Basement, Northwestern Contemporary': 1,
 '2 Stories with Basement, Other (See Remarks)': 1,
 '2 Stories with Basement, Traditional': 1,
 '2 Stories with Basement, Tudor': 1,
 '2 Stories with Basement, Victorian': 1,
 '2 Story': 0,
 '2 Story, Cape Cod': 0,
 '2 Story, Contemporary': 0,
 '2 Story, Craftsman': 0,
 '2 Story, Modern': 0,
 '2 Story, Northwestern Contemporary': 0,
 '2 Story, Other (See Remarks)': 0,
 '2 Story, Spanish/Southwestern': 0,
 '2 Story, Traditional': 0,
 '4-Plex': 0,
 '5-9 Units': 0,
 'Co-op': 0,
 'Condominium (2 Levels)': 0,
 'Condominium (2 Levels), Contemporary': 0,
 'Condominium (2 Levels), Loft': 0,
 'Condominium (2 Levels), Modern': 0,
 'Condominium (2 Levels), Townhouse': 0,
 'Condominium (2 Levels), Traditional': 0,
 'Condominium (3+ Levels)': 0,
 'Condominium (3+ Levels), Contemporary': 0,
 'Condominium (3+ Levels), Modern': 0,
 'Condominium (3+ Levels), Townhouse': 0,
 'Condominium (Single Level)': 0,
 'Condominium (Single Level), Contemporary': 0,
 'Condominium (Single Level), Craftsman': 0,
 'Condominium (Single Level), Loft': 0,
 'Condominium (Single Level), Modern': 0,
 'Condominium (Single Level), Other (See Remarks)': 0,
 'Condominium (Single Level), Spanish/Southwestern': 0,
 'Condominium (Single Level), Studio': 0,
 'Condominium (Single Level), Traditional': 0,
 'Condominium (Single Level), Tudor': 0,
 'Duplex': 0,
 'Houseboat, Cabin': 0,
 'Houseboat, Contemporary': 0,
 'Manufactured Double-Wide': 0,
 'Multi-Family': 0,
 'Multi-Level': 0,
 'Multi-Level, Contemporary': 0,
 'Multi-Level, Craftsman': 0,
 'Multi-Level, Modern': 0,
 'Multi-Level, Northwestern Contemporary': 0,
 'Multi-Level, Other (See Remarks)': 0,
 'Multi-Level, Traditional': 0,
 'Multi-Level, Tudor': 0,
 'Multi-Level, Victorian': 0,
 'Residential (1+ Acre)': 0,
 'Residential (<1 Acre)': 0,
 'Single Family Residential': 0,
 'Split-Entry': 0,
 'Split-Entry, Contemporary': 0,
 'Split-Entry, Craftsman': 0,
 'Split-Entry, Modern': 0,
 'Split-Entry, Northwestern Contemporary': 0,
 'Split-Entry, Other (See Remarks)': 0,
 'Split-Entry, Traditional': 0,
 'Townhouse': 0,
 'Townhouse, Contemporary': 0,
 'Townhouse, Craftsman': 0,
 'Townhouse, Modern': 0,
 'Townhouse, Northwestern Contemporary': 0,
 'Townhouse, Townhouse': 0,
 'Townhouse, Traditional': 0,
 'Tri-Level': 0,
 'Tri-Level, Cape Cod': 0,
 'Tri-Level, Contemporary': 0,
 'Tri-Level, Craftsman': 0,
 'Tri-Level, Modern': 0,
 'Tri-Level, Northwestern Contemporary': 0,
 'Tri-Level, Other (See Remarks)': 0,
 'Tri-Level, Traditional': 0,
 'Triplex': 0}


In [ ]:
# WARNING: dicts are only comprehensize for dataset scraped; check dataframe after mapping
listings_data['type'] = listings_data['style'].map(type_dict)
listings_data['basement'] = listings_data['style'].map(basement_dict)
converted_data = listings_data
converted_data['price'] = pd.to_numeric(converted_data['price'], 'coerce')
converted_data['beds'] = pd.to_numeric(converted_data['beds'], 'coerce')
converted_data['baths'] = pd.to_numeric(converted_data['baths'], 'coerce')
converted_data['lot'] = pd.to_numeric(converted_data['lot'], 'coerce')
converted_data['brok'] = pd.to_numeric(converted_data['brok'], 'coerce')
converted_data['age'] = pd.to_datetime(converted_data['age'], 'coerce')
converted_data['sold'] = pd.to_datetime(converted_data['sold'], 'coerce')
converted_data['size'] = pd.to_numeric(converted_data['size'], 'coerce')
# Dropping information not used/needed for model; all listings in this set were status == 'sold'
converted_data.drop(columns = ['address', 'comm', 'style', 'status', 'url', 'park'], inplace = True)
# Mask out plexes, houseboats, etc.
blank_type_mask = (converted_data['type'] != '')
# Mask out missing values for critical values
critical_vals_mask = converted_data['price'].notnull() & converted_data['beds'].notnull() & converted_data['baths'].notnull() & converted_data['size'].notnull()
converted_data = converted_data[blank_type_mask & critical_vals_mask

In [ ]:
# age is really when built; need to get a years from then to now for actual age
def calc_house_age(build_date):
    return dt.datetime.today().year - build_date.year
converted_data['age'] = converted_data['age'].apply(calc_house_age)

# Replace null brok with mode (is by FAR the most common value)
converted_data['brok'] = converted_data['brok'].fillna(3.0, inplace = True)

# Deal with tiny-lot houses (actually in acres) and null-lot townhouses/condos (actually 0)
converted_data.loc[((converted_data.type == 'house') & (converted_data.lot < 1)), 'lot'] = converted_data['lot']*43560
converted_data.loc[((converted_data['type'] == 'townhouse') | (converted_data['type'] == 'condo')) & converted_data['lot'].isnull(), 'lot'] = 0

# Moving ZIP to quintiles of mean ppsqft (approx. equal # listings)
zip_quintiles = {98101: 5, 98121: 5, 98164: 5, 98102: 5, 98109: 5, 98122: 5, 98119: 5, 
                 98107: 4, 98155: 4, 98112: 4, 98104: 4, 98103: 4, 98105: 4, 
                 98115: 3, 98144: 3, 98116: 3, 98177: 3, 98199: 3, 
                 98117: 2, 98136: 2, 98126: 2, 98133: 2, 98125: 2, 
                 98108: 1, 98166: 1, 98118: 1, 98106: 1, 98134: 1, 98146: 1, 98188: 1, 98178: 1, 98168: 1}
converted_data_simple['ZIP'] = converted_data_simple['ZIP'].map(zip_quintiles)

# Age of sale in years
def sale_age_conversion(datetime):
    return (dt.datetime.today() - datetime).days/365
converted_data_simple['sale_age'] = converted_data_simple['sold'].map(sale_age_conversion)
converted_data_simple.drop(columns = ['sold'], inplace = True)

In [ ]:
listing_plot = sbn.pairplot(model_data)

In [ ]:
model_data = converted_data_simple
model_data = pd.get_dummies(converted_data_simple, prefix = 't', columns = ['type'], drop_first = True)
sbn.heatmap(model_data.corr(), cmap="seismic", annot=True, vmin=-1, vmax=1)
plt.gca().set_ylim(len(model_data.corr())+0.5, -0.5);

In [ ]:
listing_plot = sbn.pairplot(model_data)

In [ ]:
# One $6M house was fine in terms of size and price but was throwing off a lot of other interactions
mansion_mask = model_data['price'] < 4000000
model_data = model_data[mansion_mask]
model_data.info()

In [ ]:
linreg = LinearRegression()
X = model_data[['ZIP', 'beds', 'baths', 'size', 'lot', 'age', 'sold', 'basement', 't_house', 't_townhouse']]
y = model_data['price']
# Splitting data into test/validate (60-20%) and holdout (20%); random states generated by random.org
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2,random_state=11)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=.25, random_state=95)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

pf = PolynomialFeatures()
X_train_poly = pf.fit_transform(X_train)

linreg_poly = LinearRegression()
linreg_poly.fit(X_train_poly, y_train)
print(linreg_poly.score(X_train_poly, y_train))
print(linred_poly.score(X_val_poly, y_val))

In [ ]:
list(zip(pf.get_feature_names(X_train.columns), linreg_poly.coef_))

In [ ]:
# Overfit somewhat; removing features using LASSO to see if I can improve performance
std = StandardScaler()
std.fit(X_train.values)
X_train_sc = std.transform(X_train.values)
X_val_sc = std.transform(X_val.values)
X_train_polysc = pf.fit_transform(X_train_sc)
X_val_polysc = pf.fit_transform(X_val_sc)

# Finding an alpha to minimize total error
alphalist = 10**(np.linspace(-2,2,200))
err_vec_val = np.zeros(len(alphalist))
err_vec_train = np.zeros(len(alphalist))

#Mean Absolute Error (MAE)
def mae(y_true, y_pred):
    return np.mean(np.abs(y_pred - y_true)) 

for i,curr_alpha in enumerate(alphalist):
    lasso = Lasso(alpha = curr_alpha)
    lasso.fit(X_train_polysc, y_train)
    val_set_pred = lasso.predict(X_val_polysc)
    err_vec_val[i] = mae(y_val, val_set_pred)

plt.plot(np.log10(alphalist), err_vec_val)

In [ ]:
# Finding an alpha to minimize total error
alphalist = 10**(np.linspace(2,4,200))
err_vec_val = np.zeros(len(alphalist))
err_vec_train = np.zeros(len(alphalist))
min_error = 100000
best_alpha = 0

for i,curr_alpha in enumerate(alphalist):
    lasso = Lasso(alpha = curr_alpha)
    lasso.fit(X_train_polysc, y_train)
    val_set_pred = lasso.predict(X_val_polysc)
    this_mae = mae(y_val, val_set_pred)
    if min_error > this_mae:
        min_error = this_mae
        best_alpha = curr_alpha
    err_vec_val[i] = this_mae

plt.plot(np.log10(alphalist), err_vec_val)

In [ ]:
lasso_opt = Lasso(alpha = best_alpha)
lasso_opt.fit(X_train_polysc, y_train)
lasso_coef_tuples = list(zip(pf.get_feature_names(X_train.columns), lasso_opt.coef_))
lasso_coef_tuples = sorted(lasso_coef_tuples, key = lambda x: abs(x[1]))
lasso_coef_tuples

I have reasons to believe that all the basic features are going to be useful for price prediction.

Basic features LASSO zeroed out:
- beds
- basement
- t_townhouse

Features to maybe add:
- size * ZIP
- size * t_townhouse
- sold^2

Features to possibly add and see if it helps:
- baths*size
- baths*lot
- size*sold
- beds^2
- age^2

1. Will add features back to various datasets (X_test, X_val, X_test)
2. Run linear regression using a list of features of only the basics; get MAE
3. Run linear regression using a list of only the basics plus six*ZIP; get MAE
4. Rinse repeat until it seems like the marginal benefit of additional features is small-to-nonexistent or until I reach the end of the lists here.

In [ ]:
def addPolyFeatures(df):
    df.loc[:, 'sizeZIP'] = df.loc[:, 'size'] * df.loc[:, 'ZIP']
    df.loc[:, 'sizeiftown'] = df.loc[:, 'size'] * df.loc[:, 't_townhouse']
    df.loc[:, 'sold^2'] = df.loc[:, 'sold']**2
    df.loc[:, 'bathssize'] = df.loc[:, 'baths'] * df.loc[:, 'size']
    df.loc[:, 'bathslot'] = df.loc[:, 'baths'] * df.loc[:, 'lot']
    df.loc[:, 'sizesold'] = df.loc[:, 'size'] * df.loc[:, 'sold']
    df.loc[:, 'beds^2'] = df.loc[:, 'beds']**2
    df.loc[:, 'age^2'] = df.loc[:, 'age']**2
    return df

X_train = addPolyFeatures(X_train)
X_val = addPolyFeatures(X_val)
X_test = addPolyFeatures(X_test)

features_list = ['ZIP', 'beds', 'baths', 'size', 'lot', 'age', 'sold', 'basement', 
                 't_house', 't_townhouse', 'sizeZIP', 'sizeiftown', 'sold^2', 
                 'bathssize', 'bathslot', 'sizesold', 'beds^2', 'age^2']
model = sm.OLS(y_train, X_train[features_list[0:10]])
fit = model.fit()
fit.summary()

In [ ]:
plt.scatter(fit.predict(), fit.resid)
plt.axhline(0, linestyle='--', color='gray')
plt.xlabel('Predicted price', fontsize=18)
plt.ylabel('Residuals', fontsize=18);

In [ ]:
def rsme(y_true, y_pred):
    return np.sqrt(np.mean((y_pred - y_true)**2))
print(mae(y_train, fit.predict()))
print(rsme(y_train, fit.predict()))

In [ ]:
mae_list = []
rsme_list = []
for i in range(10, 19):
    model = sm.OLS(y_train, sm.add_constant(X_train[features_list[0:i]]))
    fit = model.fit()
    prediction = fit.predict(sm.add_constant(X_val[features_list[0:i]]))
    mae_list.append(mae(y_val, prediction))
    rsme_list.append(rsme(y_val, prediction))

plt.plot(range(10, 19), mae_list, color = 'red')
plt.plot(range(10, 19), rsme_list, color = 'blue')
plt.xlabel('number of features')
plt.ylabel('House price error ($)')
plt.title('Error reduction as polynomial features added')

Did try subtracting the feature that didn't help; worse results. Discussed with Brian who pointed out that this approach is sensitive to the order of feature addition. LASSO again to see if I can reduce the complexity while preserving sensibility.

In [ ]:
X_train_sc = std.transform(X_train.values)
X_val_sc = std.transform(X_val.values)
# Finding an alpha to minimize total error
alphalist = 10**(np.linspace(-2,2,200))
err_vec_val = np.zeros(len(alphalist))
err_vec_train = np.zeros(len(alphalist))
min_error = 100000
best_alpha = 0

for i,curr_alpha in enumerate(alphalist):
    lasso = Lasso(alpha = curr_alpha)
    lasso.fit(X_train_sc, y_train)
    val_set_pred = lasso.predict(X_val_sc)
    this_mae = mae(y_val, val_set_pred)
    if min_error > this_mae:
        min_error = this_mae
        best_alpha = curr_alpha
    err_vec_val[i] = this_mae

plt.plot(np.log10(alphalist), err_vec_val)

In [ ]:
lasso = Lasso(best_alpha)
lasso.fit(X_train_sc, y_train)
lasso_coef_tuples = list(zip(X_train.columns, lasso_opt.coef_))
lasso_coef_tuples = sorted(lasso_coef_tuples, key = lambda x: abs(x[1]))
lasso_coef_tuples

In [ ]:
features_list3 = ['lot', 'beds', 'bathslot', 'size', 'basement', 't_townhouse', 'sold', 'beds^2', 'age^2', 'sizesold', 'age']
model = sm.OLS(y_train, X_train[features_list3])
fit = model.fit()
fit.summary()

In [ ]:
plt.scatter(fit.predict(), fit.resid)
plt.axhline(0, linestyle='--', color='gray')
plt.xlabel('Predicted price', fontsize=18)
plt.ylabel('Residuals', fontsize=18);

In [ ]:
print(mae(y_train, fit.predict()))
print(rsme(y_train, fit.predict()))

# Modeling can't capture the intangibles
This model is rather worse than the more complicated model. Fundamentally, I didn't capture some of the features that are likely contributing substantially to the final sold price of a home (view, newness of interior, curb appeal, etc.). 

Instead of trying to figure out how much a house should be listed for, let's instead try to capture how much different first order features contribute to a house's sale price. Step one will be running LASSO on X_train2 (unchewed data set) to see if there are any features that should be tossed out and see if we agree with that.

In [ ]:
# Getting this going again without overwriting model or train/val sets I want to use again.
X_train_val2, X_test2, y_train_val2, y_test2 = train_test_split(X, y, test_size=0.2,random_state=93)
X_train2, X_val2, y_train2, y_val2 = train_test_split(X_train_val2, y_train_val2, test_size=.25, random_state=98)
std = StandardScaler()
std.fit(X_train2)
X_train2_sc = std.transform(X_train2.values)
X_val2_sc = std.transform(X_val2.values)

# Finding an alpha to minimize total error
alphalist = 10**(np.linspace(-2,2,200))
err_vec_val = np.zeros(len(alphalist))
err_vec_train = np.zeros(len(alphalist))
min_error = 100000
best_alpha = 0

for i,curr_alpha in enumerate(alphalist):
    lasso = Lasso(alpha = curr_alpha)
    lasso.fit(X_train2_sc, y_train2)
    val_set_pred = lasso.predict(X_val2_sc)
    this_mae = mae(y_val2, val_set_pred)
    if min_error > this_mae:
        min_error = this_mae
        best_alpha = curr_alpha
    err_vec_val[i] = this_mae

plt.plot(np.log10(alphalist), err_vec_val)

In [ ]:
# Finding an alpha to minimize total error
alphalist = 10**(np.linspace(2,4,200))
err_vec_val = np.zeros(len(alphalist))
err_vec_train = np.zeros(len(alphalist))
min_error = 100000
best_alpha = 0

for i,curr_alpha in enumerate(alphalist):
    lasso = Lasso(alpha = curr_alpha)
    lasso.fit(X_train2_sc, y_train2)
    val_set_pred = lasso.predict(X_val2_sc)
    this_mae = mae(y_val2, val_set_pred)
    if min_error > this_mae:
        min_error = this_mae
        best_alpha = curr_alpha
    err_vec_val[i] = this_mae

plt.plot(np.log10(alphalist), err_vec_val)

In [ ]:
lasso = Lasso(best_alpha)
lasso.fit(X_train2_sc, y_train2)
lasso_coef_tuples = list(zip(X_train2.columns, lasso_opt.coef_))
lasso_coef_tuples = sorted(lasso_coef_tuples, key = lambda x: abs(x[1]))
lasso_coef_tuples

This is interesting! Basements are actually a minus (finished basement sqft worth less than main floor sqft, mayhaps?). Sold date works exactly as I'd expect, a discount for times in the past to account for inflation and increases in house prices.

Definitely surprised that location and number of baths don't matter! I wonder how much of that is homogeneity in "niceness/richness" of neighborhood within a given ZIP and how much is just that relatively little variation is accounted for by ZIP alone (i.e. houses in nicer areas also tend to be larger). That said, I think that it's important for my purposes to keep bath in; ZIP can go. We'll see what house type does after that?

In [ ]:
# Linear regression on final features list using train/val 80% chunk
features_list4 = ['beds', 'baths', 'size', 'age', 'sold', 'lot', 'basement', 't_house', 't_townhouse']
model = sm.OLS(y_train_val2, X_train_val2[features_list4])
fit = model.fit()
fit.summary()

In [ ]:
# Very much leaning on https://zhiyzuo.github.io/Python-Plot-Regression-Coefficient/ to graph coefficients w/ CI
error_bars = fit.params - fit.conf_int()[0]
coef_df = pd.DataFrame({'coefficient': fit.params.values,
                       'error': error_bars.values,
                       'variable': error_bars.index.values})
fig, ax = plt.subplots()
coef_df.plot(x = 'variable', y = 'coefficient', kind = 'bar', ax = ax, yerr = 'error')
ax.set_xlabel('')
ax.axhline(y=0, linestyle='--', color='black', linewidth=1)